In [1]:
from huggingface_hub import InferenceClient
from tqdm import tqdm
import os
import json

In [2]:
client = InferenceClient(
    "https://mixtral.us-west-2.mesolitica.com/"
)


def format_prompt(message, history):
  prompt = "<s>"
  for user_prompt, bot_response in history:
    prompt += f"[INST] {user_prompt} [/INST]"
    prompt += f" {bot_response}</s> "
  prompt += f"[INST] {message} [/INST]"
  return prompt

In [3]:
generate_kwargs = dict(
    temperature=1.0,
    max_new_tokens=5120,
    top_p=0.95,
    repetition_penalty=1.0,
    do_sample=True,
)

In [4]:
ls = []
with open('filtered-data-analytics-0.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        ls.append(l)
len(ls)

87190

In [5]:
!mkdir filtered-data-analytics-0
# !rm filtered-data-analytics-0/*

mkdir: cannot create directory ‘filtered-data-analytics-0’: File exists


In [ ]:
for i in tqdm(range(len(ls))):
    filename = os.path.join('filtered-data-analytics-0', f'{i}.json')
    if os.path.exists(filename):
        continue
    prompt = """
Please gain inspiration from the following random code snippet to create a high-quality programming problem. Present your output in two distinct sections: [Problem Description] and [Solution].

Code snippet for inspiration:
```
{code}
```

Guidelines for each section:

1. [Problem Description]: This should be **completely self-contained**, providing all the contextual information one needs to understand and solve the problem. Assume common programming knowledge, but ensure that any specific context, variables, or code snippets pertinent to this problem are explicitly included.

2. [Solution]: Offer a comprehensive, **correct** solution that accurately addresses the [Problem Description] you provided.
""".strip()

    new_text = prompt.format(code = ls[i]['content'].strip())
    formatted_prompt = format_prompt(new_text, [])
    try:
        stream = client.text_generation(formatted_prompt, **generate_kwargs, stream=False, details=True, return_full_text=False)
        with open(filename, 'w') as fopen:
            fopen.write(f'{json.dumps(stream.generated_text.strip())}\n')
    except:
        pass

  1%|▏         | 1108/87190 [11:45:05<993:03:18, 41.53s/it] 

In [1]:
from glob import glob
import json

files = glob('filtered-data-analytics-0/*.json')

In [6]:
with open(files[4]) as fopen:
    print(json.load(fopen))

**[Problem Description]**

A script is provided to gather and organize benchmark results for deep learning model training and inference using different hardware configurations and precision settings. The script looks for logs in the specified path, and organizes the results into CSV files for both training and inference throughput, as well as training batch size. The logs are organized based on a naming convention that maps to different hardware systems and GPU counts.

However, the script currently only supports a predefined list of models (`list_test`) and hardware systems (`list_system`). Your task is to modify the script to make it more flexible and configurable, so that users can easily add new models, hardware systems, and other variables to be benchmarked.

**[Problem Requirements]**

1. Make the `list_test` and `list_system` variables configurable by users.
2. Add support for users to specify their own list of models and hardware systems.
3. Allow users to easily add new variab